# Использование инструментов Яндекс Облака для анализа данных Яндекс Метрики
## Logs API

`Logs API` позволяет выгрузить сырые данные со счетчика.

Документация по Logs API - https://yandex.ru/dev/metrika/doc/api2/logs/intro.html

Данные для этого кейса также доступны на Яндекс.Диске - !!!!!https://yadi.sk/d/XJMDdTDmDO-c9g!!!!!

### Шаг 1: получаем токен
Для работы с API необходимо получить свой токен - https://yandex.ru/dev/oauth/doc/dg/tasks/get-oauth-token.html

Создаем приложение тут (указываем права для чтения в Яндекс.Метрике) - https://oauth.yandex.ru/client/new

Переходим по ссылке вида - `https://oauth.yandex.ru/authorize?response_type=token&client_id=<идентификатор приложения>`

Полученный токен можно сохранить в домашнюю директорию в файл `.yatoken.txt`

In [166]:
with open('../.yatoken.txt') as f:
    TOKEN = f.read().strip()

/kernel/lib/python3.7/site-packages/ml_kernel/kernel.py:818: UserWarning: The following variables cannot be serialized: f
  warnings.warn(message)


### Шаг 2: проверяем, можно ли создать запрос в Logs API

In [167]:
import requests
import pandas as pd
from io import StringIO
import datetime
import json
from urllib.parse import urlencode
import time

In [168]:
API_HOST = 'https://api-metrika.yandex.ru'
COUNTER_ID = 73226638
START_DATE = '2020-07-01'
END_DATE = '2020-09-30'
SOURCE = 'hits'
API_FIELDS = ('ym:pv:date', 'ym:pv:dateTime', 'ym:pv:URL', 'ym:pv:deviceCategory', 
         'ym:pv:operatingSystemRoot', 'ym:pv:clientID', 'ym:pv:browser', 'ym:pv:lastTrafficSource')


In [169]:
header_dict = {'Authorization': f'OAuth {TOKEN}',
'Content-Type': 'application/x-yametrika+json'
}

In [170]:
url_params = urlencode(
    [
        ('date1', START_DATE),
        ('date2', END_DATE),
        ('source', SOURCE),
        ('fields', ','.join(API_FIELDS))
    ]
)

url = '{host}/management/v1/counter/{counter_id}/logrequests/evaluate?'\
    .format(host=API_HOST, counter_id=COUNTER_ID) + url_params

r = requests.get(url, headers = header_dict)

In [171]:
r.status_code

200

In [172]:
json.loads(r.text)['log_request_evaluation']

{'possible': True, 'max_possible_day_quantity': 2125}

### Шаг 3: создаем запрос

In [173]:
url_params = urlencode(
    [
        ('date1', START_DATE),
        ('date2', END_DATE),
        ('source', SOURCE),
        ('fields', ','.join(sorted(API_FIELDS, key=lambda s: s.lower())))
    ]
)
url = '{host}/management/v1/counter/{counter_id}/logrequests?'\
    .format(host=API_HOST,
            counter_id=COUNTER_ID) \
      + url_params

r = requests.post(url, headers=header_dict)

In [174]:
r.status_code

200

In [175]:
json.loads(r.text)['log_request']

{'request_id': 15522244,
 'counter_id': 73226638,
 'source': 'hits',
 'date1': '2020-07-01',
 'date2': '2020-09-30',
 'fields': ['ym:pv:browser',
  'ym:pv:clientID',
  'ym:pv:date',
  'ym:pv:dateTime',
  'ym:pv:deviceCategory',
  'ym:pv:lastTrafficSource',
  'ym:pv:operatingSystemRoot',
  'ym:pv:URL'],
 'status': 'created',
 'attribution': 'LASTSIGN'}

In [176]:
request_id = json.loads(r.text)['log_request']['request_id']

In [177]:
request_id

15522244

### Шаг 4: ждем окончания обработки

In [178]:
status = 'created'
while status == 'created':
    time.sleep(60)
    print('trying')
    url = '{host}/management/v1/counter/{counter_id}/logrequest/{request_id}' \
            .format(request_id=request_id,
                    counter_id=COUNTER_ID,
                    host=API_HOST)

    r = requests.get(url, headers=header_dict)
    if r.status_code == 200:
        status = json.loads(r.text)['log_request']['status']
        print(json.dumps(json.loads(r.text)['log_request'], indent = 4))
    else:
        raise(BaseException(r.text))

trying
{
    "request_id": 15522244,
    "counter_id": 73226638,
    "source": "hits",
    "date1": "2020-07-01",
    "date2": "2020-09-30",
    "fields": [
        "ym:pv:browser",
        "ym:pv:clientID",
        "ym:pv:date",
        "ym:pv:dateTime",
        "ym:pv:deviceCategory",
        "ym:pv:lastTrafficSource",
        "ym:pv:operatingSystemRoot",
        "ym:pv:URL"
    ],
    "status": "processed",
    "size": 348454420,
    "parts": [
        {
            "part_number": 0,
            "size": 348454420
        },
        {
            "part_number": 1,
            "size": 0
        },
        {
            "part_number": 2,
            "size": 0
        },
        {
            "part_number": 3,
            "size": 0
        },
        {
            "part_number": 4,
            "size": 0
        },
        {
            "part_number": 5,
            "size": 0
        },
        {
            "part_number": 6,
            "size": 0
        }
    ],
    "attribution": "LAS

In [179]:
json.loads(r.text)['log_request']

{'request_id': 15522244,
 'counter_id': 73226638,
 'source': 'hits',
 'date1': '2020-07-01',
 'date2': '2020-09-30',
 'fields': ['ym:pv:browser',
  'ym:pv:clientID',
  'ym:pv:date',
  'ym:pv:dateTime',
  'ym:pv:deviceCategory',
  'ym:pv:lastTrafficSource',
  'ym:pv:operatingSystemRoot',
  'ym:pv:URL'],
 'status': 'processed',
 'size': 348454420,
 'parts': [{'part_number': 0, 'size': 348454420},
  {'part_number': 1, 'size': 0},
  {'part_number': 2, 'size': 0},
  {'part_number': 3, 'size': 0},
  {'part_number': 4, 'size': 0},
  {'part_number': 5, 'size': 0},
  {'part_number': 6, 'size': 0}],
 'attribution': 'LASTSIGN'}

In [180]:
parts = json.loads(r.text)['log_request']['parts']
parts

[{'part_number': 0, 'size': 348454420},
 {'part_number': 1, 'size': 0},
 {'part_number': 2, 'size': 0},
 {'part_number': 3, 'size': 0},
 {'part_number': 4, 'size': 0},
 {'part_number': 5, 'size': 0},
 {'part_number': 6, 'size': 0}]

### Шаг 5: выгружаем данные

In [181]:
tmp_dfs = []
for part_num in map(lambda x: x['part_number'], parts):
    url = '{host}/management/v1/counter/{counter_id}/logrequest/{request_id}/part/{part}/download' \
            .format(
                host=API_HOST,
                counter_id=COUNTER_ID,
                request_id=request_id,
                part=part_num
            )

    r = requests.get(url, headers=header_dict)
    if r.status_code == 200:
        tmp_df = pd.read_csv(StringIO(r.text), sep = '\t')
        tmp_dfs.append(tmp_df)
    else:
        raise(BaseError(r.text))
        
logs_df = pd.concat(tmp_dfs)

In [182]:
logs_df.shape

(3037737, 8)

In [183]:
logs_df.to_csv('matemarketing_case_data.csv', sep = '\t', index = False)

## ClickHouse
### Подключение и настройка
https://cloud.yandex.ru/docs/managed-clickhouse/
(см. слайды)

### Данные для доступа
* Из интерфейса облака в разделе хосты копируем имя хост в переменную `CH_HOST` вида `'https://{ИМЯ_ХОСТА}.mdb.yandexcloud.net:8443'`
* Используем заведенного юзера в переменной `CH_USER`
* Сохраним пароль заведенного пользователя в текстовый файл `.chpass.txt`
* В переменную `CH_PASS` считаем содержимое файла `.chpass.txt`
* В переменную `cacert` поместим путь к сертификату для подключения к серверу. Файл `YandexInternalRootCA.crt` должен лежать в репозитории

In [186]:
CH_HOST = 'https://rc1a-rt82aoo61ovjyuxf.mdb.yandexcloud.net:8443'
CH_USER = 'z_sergey'

with open('../.chpass.txt') as f:
    CH_PASS = f.read().strip()

cacert = 'YandexInternalRootCA.crt'


/kernel/lib/python3.7/site-packages/ml_kernel/kernel.py:818: UserWarning: The following variables cannot be serialized: f
  warnings.warn(message)


### Проверяем ClickHouse
Используя заговленные выше переменные проверим доступ до сервера (как в документации https://cloud.yandex.ru/docs/managed-clickhouse/operations/connect#connection-string)

При успешном подключении не произойдет никакой ошибки при выполнении кода ниже, а в `rs.text` будет содержаться версия сервера ClickHouse (например `20.8.12.2`)

In [189]:
url = '{host}/?database={db}&query={query}'.format(
        host=CH_HOST,
        db='default',
        query='SELECT version()')

auth = {
        'X-ClickHouse-User': CH_USER,
        'X-ClickHouse-Key': CH_PASS,
    }

rs = requests.get(url, headers=auth, verify=cacert)
# 
rs.raise_for_status()

print(rs.text)


20.8.12.2



###  Функции для интеграции с ClickHouse

Напишем функции для интеграции с ClickHouse: первая функция просто возвращает результат из DataBase, вторая же преобразует его в pandas DataFrame.

Также напишем сразу удобную функцию для загрузки данных.

In [190]:
def get_clickhouse_data(query, host = CH_HOST, connection_timeout = 1500):
    r = requests.post(host, params = {'query': query, 'user': CH_USER, 'password':CH_PASS}, timeout = connection_timeout, verify=cacert)
    if r.status_code == 200:
        return r.text
    else:
        raise ValueError(r.text)
        
def get_clickhouse_df(query, host = CH_HOST, connection_timeout = 1500):
    data = get_clickhouse_data(query, host, connection_timeout) 
    df = pd.read_csv(StringIO(data), sep = '\t')
    return df

def upload(table, content, host=CH_HOST):
    content = content.encode('utf-8')
    query_dict = {
            'query': 'INSERT INTO ' + table + ' FORMAT TabSeparatedWithNames ',
            'user': CH_USER, 
            'password':CH_PASS
        }
    r = requests.post(host, data=content, params=query_dict, verify=cacert)
    result = r.text
    if r.status_code == 200:
        return result
    else:
        raise ValueError(r.text)

### Загружаем данные

In [191]:
logs_df.head()

,ym:pv:browser,ym:pv:clientID,ym:pv:date,ym:pv:dateTime,ym:pv:deviceCategory,ym:pv:lastTrafficSource,ym:pv:operatingSystemRoot,ym:pv:URL
0,firefox,2787558532729551569,2020-07-27,2020-07-27 12:19:54,1,organic,gnu_linux,https://supermarket.ru/other
1,firefox,2787558532729551569,2020-07-27,2020-07-27 12:20:19,1,organic,gnu_linux,https://supermarket.ru/other
2,edge,17980493159973083792,2020-07-27,2020-07-27 09:27:50,1,organic,windows,https://supermarket.ru/product_8196
3,edge,17980493159973083792,2020-07-27,2020-07-27 09:28:06,1,direct,windows,https://supermarket.ru/product_8196
4,edge,17980493159973083792,2020-07-27,2020-07-27 09:33:46,1,internal,windows,https://supermarket.ru/basket


In [192]:
logs_df.rename(columns={'ym:pv:browser':'Browser',
                'ym:pv:clientID':'ClientID',
                'ym:pv:date':'EventDate',
                'ym:pv:dateTime':'EventTime',
                'ym:pv:deviceCategory':'DeviceCategory',
                'ym:pv:lastTrafficSource':'TraficSource',
                'ym:pv:operatingSystemRoot':'OSRoot',
                'ym:pv:URL':'URL'}, inplace = True)

In [193]:
q = 'drop table if exists metrica_data.hits '
get_clickhouse_data(q)

q = '''
create table metrica_data.hits (
    Browser String,
    ClientID UInt64,
    EventDate Date,
    EventTime DateTime,
    DeviceCategory String,
    TraficSource String,
    OSRoot String,
    URL String
) ENGINE = MergeTree(EventDate, intHash32(ClientID), (EventDate, intHash32(ClientID)), 8192)
'''

get_clickhouse_data(q)

''

In [194]:
upload(
    'metrica_data.hits',
    logs_df.to_csv(index = False, sep = '\t'))

''

### Разминка: смотрим на пути пользователей

In [195]:
q = '''
SELECT
    URL,
    uniq(ClientID) as users,
    count() as hits
FROM metrica_data.hits
GROUP BY URL
ORDER BY users DESC
LIMIT 10
FORMAT TabSeparatedWithNames
'''

get_clickhouse_df(q)

,URL,users,hits
0,https://supermarket.ru/,40191,167248
1,https://supermarket.ru/search,31297,376060
2,https://supermarket.ru/promo_actions,22684,93260
3,https://supermarket.ru/product_1174,20912,37002
4,https://supermarket.ru/catalog,13424,60112
5,https://supermarket.ru/basket,13324,196363
6,https://supermarket.ru/info,10506,20268
7,https://supermarket.ru/account,10482,59975
8,https://supermarket.ru/promo_action_2068,10335,15853
9,https://supermarket.ru/other,10222,87043


In [196]:
q = '''
SELECT
    ClientID,
    groupArray(URL) as path,
    arraySlice(groupArray(URL), 1, 5) as path_lim
FROM
    (SELECT * FROM metrica_data.hits ORDER BY ClientID, EventTime)
GROUP BY ClientID
LIMIT 10
FORMAT TabSeparatedWithNames
'''

get_clickhouse_df(q)

,ClientID,path,path_lim
0,2773878518280827494,['https://supermarket.ru/promo_action_2068'],['https://supermarket.ru/promo_action_2068']
1,11346328702551058579,"['https://supermarket.ru/product_8046','https:...","['https://supermarket.ru/product_8046','https:..."
2,13106442848447404077,"['https://supermarket.ru/login','https://super...","['https://supermarket.ru/login','https://super..."
3,13041872533683226559,"['https://supermarket.ru/product_1858','https:...","['https://supermarket.ru/product_1858','https:..."
4,14051588439054376991,"['https://supermarket.ru/product_1036','https:...","['https://supermarket.ru/product_1036','https:..."
5,1880557729445743004,['https://supermarket.ru/product_2556'],['https://supermarket.ru/product_2556']
6,17287937153480005918,"['https://supermarket.ru/','https://supermarke...","['https://supermarket.ru/','https://supermarke..."
7,1717899126568995497,"['https://supermarket.ru/product_1174','https:...","['https://supermarket.ru/product_1174','https:..."
8,12735858831012285246,"['https://supermarket.ru/','https://supermarke...","['https://supermarket.ru/','https://supermarke..."
9,16757513062212456860,"['https://supermarket.ru/','https://supermarke...","['https://supermarket.ru/','https://supermarke..."


In [197]:
q = '''
SELECT
    path_lim,
    count() as users
FROM
    (SELECT
        ClientID,
        arraySlice(groupArray(URL), 1, 5) as path_lim
    FROM
        (SELECT * FROM metrica_data.hits ORDER BY ClientID, EventTime)
    GROUP BY ClientID)
GROUP BY path_lim
ORDER BY users desc
LIMIT 10
FORMAT TabSeparatedWithNames
'''

get_clickhouse_df(q)

,path_lim,users
0,['https://supermarket.ru/product_1174'],7931
1,"['https://supermarket.ru/product_1174','https:...",6827
2,['https://supermarket.ru/promo_action_2068'],6717
3,"['https://supermarket.ru/','https://supermarke...",6005
4,"['https://supermarket.ru/search','https://supe...",3918
5,['https://supermarket.ru/promo_actions'],3582
6,"['https://supermarket.ru/promo_actions','https...",3426
7,['https://supermarket.ru/promo_action_3710'],3374
8,['https://supermarket.ru/'],3304
9,['https://supermarket.ru/search'],3253


Пофильтруем повторяющиеся шаги в путях, чтобы было понятнее, в каких разделах были пользователи

In [198]:
q = '''
SELECT
    path,
    count() as users,
    arrayEnumerate(path) as indexes,
    arrayFilter(val, index -> (index = 1) or (path[index - 1] != path[index]),
        path, indexes) as path_filt
FROM
    (SELECT
        ClientID,
        groupArray(URL) as path
    FROM
        (SELECT * FROM metrica_data.hits ORDER BY ClientID, EventTime)
    GROUP BY ClientID)
GROUP BY path
ORDER BY users desc
LIMIT 10
FORMAT TabSeparatedWithNames
'''

get_clickhouse_df(q)

ValueError: Code: 241, e.displayText() = DB::Exception: Memory limit (for query) exceeded: would use 4.55 GiB (attempt to allocate chunk of 4294969752 bytes), maximum: 3.00 GiB (version 20.8.12.2 (official build))


In [137]:
q = '''
SELECT 
    path_filt,
    sum(users) as users
FROM
    (SELECT
        path,
        count() as users,
        arrayEnumerate(path) as indexes,
        arrayFilter(val, index -> (index = 1) or (path[index - 1] != path[index]),
            path, indexes) as path_filt
    FROM
        (SELECT
            ClientID,
            groupArray(URL) as path
        FROM
            (SELECT * FROM metrica_data.hits ORDER BY ClientID, EventTime)
        GROUP BY ClientID)
    GROUP BY path)
GROUP BY path_filt
ORDER BY users DESC
LIMIT 10
FORMAT TabSeparatedWithNames
'''

get_clickhouse_df(q)

## Расчет funnels

#### Подход в лоб

In [138]:
q = '''
SELECT
    ClientID,
    max(URL = 'https://supermarket.ru/main') as step1_achieved,
    sequenceMatch('(?1).*(?2)')(EventTime, URL = 'https://supermarket.ru/main', 
      URL = 'https://supermarket.ru/catalog') as step2_achieved,
    sequenceMatch('(?1).*(?2).*(?3)')(EventTime, URL = 'https://supermarket.ru/main', 
      URL = 'https://supermarket.ru/catalog', URL = 'https://supermarket.ru/product') as step3_achieved
FROM metrica_data.hits
GROUP BY ClientID
LIMIT 10
FORMAT TabSeparatedWithNames
'''

get_clickhouse_df(q)

In [139]:
q = '''
SELECT
    sum(step1_achieved) as step1,
    sum(step2_achieved) as step2,
    sum(step3_achieved) as step3
    --round(100.*step2/step1, 2) as step2_to_1,
    --round(100.*step3/step1, 2) as step3_to_1
FROM
    (SELECT
        ClientID,
        max(URL = 'https://supermarket.ru/main') as step1_achieved,
        sequenceMatch('(?1).*(?2)')(EventTime, URL = 'https://supermarket.ru/main', 
          URL = 'https://supermarket.ru/catalog') as step2_achieved,
        sequenceMatch('(?1).*(?2).*(?3)')(EventTime, URL = 'https://supermarket.ru/main', 
          URL = 'https://supermarket.ru/catalog', URL = 'https://supermarket.ru/product') as step3_achieved
    FROM metrica_data.hits
    GROUP BY ClientID)
FORMAT TabSeparatedWithNames
'''

fdf = get_clickhouse_df(q)

In [140]:
fdf

Визуализируем

In [141]:
from plotly.offline import init_notebook_mode, iplot
import plotly
import plotly.graph_objs as go

init_notebook_mode(connected=True)

In [142]:
colors = colors = ['#d54936', '#faca34', '#437cba', '#8bc34a', '#795548', '#309688', '#000000', '#40bcd4', '#9e9e9e', '#3ca9f4']
phases = ['Main', 'Catalog', 'Product']
values = fdf.iloc[0].values

In [144]:
def plot_funnel(phases, values):
    n_phase = len(phases)
    plot_width = 500.

    # height of a section and difference between sections 
    section_h = 100
    section_d = 10

    # multiplication factor to calculate the width of other sections
    unit_width = plot_width / max(values)

    # width of each funnel section relative to the plot width
    phase_w = [int(value * unit_width) for value in values]
    print (phase_w)

    # plot height based on the number of sections and the gap in between them
    height = section_h * n_phase + section_d * (n_phase - 1)
    
    # list containing all the plot shapes
    shapes = []

    # list containing the Y-axis location for each section's name and value text
    label_y = []

    for i in range(n_phase):
            if (i == n_phase-1):
                    points = [phase_w[i] / 2, height, phase_w[i] / 2, height - section_h]
            else:
                    points = [phase_w[i] / 2, height, phase_w[i+1] / 2, height - section_h]

            path = 'M {0} {1} L {2} {3} L -{2} {3} L -{0} {1} Z'.format(*points)

            shape = {
                    'type': 'path',
                    'path': path,
                    'fillcolor': colors[i],
                    'line': {
                        'width': 1,
                        'color': colors[i]
                    }
            }
            shapes.append(shape)

            # Y-axis location for this section's details (text)
            label_y.append(height - (section_h / 2))

            height = height - (section_h + section_d)

    # For phase names
    label_trace = go.Scatter(
        x=[-350]*n_phase,
        y=label_y,
        mode='text',
        text=phases,
        textfont=dict(
            color='rgb(40,40,40)',
            size=15
        )
    )

    # For phase values
    value_trace = go.Scatter(
        x=[350]*n_phase,
        y=label_y,
        mode='text',
        text=values,
        textfont=dict(
            color='rgb(40,40,40)',
            size=15
        )
    )

    data = [label_trace, value_trace]

    layout = go.Layout(
        title="<b>Funnel Chart</b>",
        titlefont=dict(
            size=20,
            color='rgb(0,0,0)'
        ),
        shapes=shapes,
        height=560,
        width=800,
        showlegend=False,
        paper_bgcolor='rgba(255,255,255,1)',
        plot_bgcolor='rgba(255,255,255,1)',
        xaxis=dict(
            showticklabels=False,
            zeroline=False,
            showgrid=False,
            range=[-450, 450]
        ),
        yaxis=dict(
            showticklabels=False,
            zeroline=False,
            showgrid=False
        )
    )

    fig = go.Figure(data=data, layout=layout)
    iplot(fig, show_link=False)

In [145]:
plot_funnel(phases, values)

### Добавим еще один шаг

In [146]:
%%time
q = '''
SELECT
    sum(step1_achieved) as step1,
    sum(step2_achieved) as step2,
    sum(step3_achieved) as step3,
    sum(step4_achieved) as step4,
    sum(step5_achieved) as step5
FROM
    (SELECT
        ClientID,
        max(URL = 'https://supermarket.ru/main') as step1_achieved,
        sequenceMatch('(?1).*(?2)')(EventTime, URL = 'https://supermarket.ru/main', 
          URL = 'https://supermarket.ru/catalog') as step2_achieved,
        sequenceMatch('(?1).*(?2).*(?3)')(EventTime, URL = 'https://supermarket.ru/main', 
          URL = 'https://supermarket.ru/catalog', URL = 'https://supermarket.ru/product') as step3_achieved,
        sequenceMatch('(?1).*(?2).*(?3).*(?4)')(EventTime, URL = 'https://supermarket.ru/main', 
          URL = 'https://supermarket.ru/catalog', URL = 'https://supermarket.ru/product', 
          URL = 'https://supermarket.ru/basket') as step4_achieved,
        sequenceMatch('(?1).*(?2).*(?3).*(?4).*(?5)')(EventTime, URL = 'https://supermarket.ru/main', 
          URL = 'https://supermarket.ru/catalog', URL = 'https://supermarket.ru/product', 
          URL = 'https://supermarket.ru/basket', URL = 'https://supermarket.ru/finish_order') as step5_achieved
    FROM metrica_data.hits
    GROUP BY ClientID)
FORMAT TabSeparatedWithNames
'''

fdf = get_clickhouse_df(q)

In [147]:
fdf

### Пойдем другим путем

In [149]:
q = '''
SELECT
    ClientID,
    groupArray(cast(EventTime as UInt64)) as times,
    groupArray(URL) as urls,
    arrayFilter(time, url -> url = 'https://supermarket.ru/main', times, urls)[1] as step1_time
FROM
    (SELECT * FROM metrica_data.hits ORDER BY ClientID, EventTime)
GROUP BY ClientID
LIMIT 5
FORMAT TabSeparatedWithNames
'''

get_clickhouse_df(q)

In [150]:
q = '''
SELECT
    ClientID,
    groupArray(cast(EventTime as UInt64)) as times,
    groupArray(URL) as urls,
    arrayFilter(time, url -> url = 'https://supermarket.ru/main', times, urls)[1] as step1_time,
    arrayFilter(time, url -> url = 'https://supermarket.ru/catalog' and step1_time != 0 and time >= step1_time, times, urls)[1] as step2_time
FROM
    (SELECT * FROM metrica_data.hits ORDER BY ClientID, EventTime)
GROUP BY ClientID
LIMIT 5
FORMAT TabSeparatedWithNames
'''

get_clickhouse_df(q)

In [151]:
q = '''
SELECT
    countIf(step1_time != 0) as step1,
    countIf(step2_time != 0) as step2,
    countIf(step3_time != 0) as step3,
    countIf(step4_time != 0) as step4,
    countIf(step5_time != 0) as step5
FROM
    (SELECT
        ClientID,
        groupArray(cast(EventTime as UInt64)) as times,
        groupArray(URL) as urls,
        arrayFilter(time, url -> url = 'https://supermarket.ru/main', times, urls)[1] as step1_time,
        arrayFilter(time, url -> url = 'https://supermarket.ru/catalog' and step1_time != 0 and time >= step1_time, times, urls)[1] as step2_time,
        arrayFilter(time, url -> url = 'https://supermarket.ru/product' and step2_time != 0 and time >= step2_time, times, urls)[1] as step3_time,
        arrayFilter(time, url -> url = 'https://supermarket.ru/basket' and step3_time != 0 and time >= step3_time, times, urls)[1] as step4_time,
        arrayFilter(time, url -> url = 'https://supermarket.ru/finish_order' and step4_time != 0 and time >= step4_time, times, urls)[1] as step5_time
    FROM
        (SELECT * FROM metrica_data.hits ORDER BY ClientID, EventTime)
    GROUP BY ClientID)
FORMAT TabSeparatedWithNames
'''

fdf = get_clickhouse_df(q)

In [152]:
phases = ['Main', 'Catalog', 'Product', 'Basket', 'Finish order']
values = fdf.iloc[0].values

plot_funnel(phases, values)

Можем также посчитать среднее время достижения шагов

In [153]:
q = '''
SELECT
    medianIf(step2_time - step1_time, step2_time != 0) as step2_from_step1_mean_time,
    medianIf(step3_time - step2_time, step3_time != 0) as step3_from_step2_mean_time,
    medianIf(step4_time - step3_time, step4_time != 0) as step4_from_step3_mean_time,
    medianIf(step5_time - step4_time, step5_time != 0) as step5_from_step4_mean_time
FROM
    (SELECT
        ClientID,
        groupArray(cast(EventTime as UInt64)) as times,
        groupArray(URL) as urls,
        arrayFilter(time, url -> url = 'https://supermarket.ru/main', times, urls)[1] as step1_time,
        arrayFilter(time, url -> url = 'https://supermarket.ru/catalog' and step1_time != 0 and time >= step1_time, times, urls)[1] as step2_time,
        arrayFilter(time, url -> url = 'https://supermarket.ru/product' and step2_time != 0 and time >= step2_time, times, urls)[1] as step3_time,
        arrayFilter(time, url -> url = 'https://supermarket.ru/basket' and step3_time != 0 and time >= step3_time, times, urls)[1] as step4_time,
        arrayFilter(time, url -> url = 'https://supermarket.ru/finish_order' and step4_time != 0 and time >= step4_time, times, urls)[1] as step5_time
    FROM
        (SELECT * FROM metrica_data.hits ORDER BY ClientID, EventTime)
    GROUP BY ClientID)
FORMAT TabSeparatedWithNames
'''

get_clickhouse_df(q)

# Воронки по браузеру

In [155]:
q = '''
SELECT
    Browser,
    countIf(step1_time != 0) as step1,
    countIf(step2_time != 0) as step2,
    countIf(step3_time != 0) as step3,
    countIf(step4_time != 0) as step4,
    countIf(step5_time != 0) as step5
FROM
(
    SELECT
        Browser,
        ClientID,
        groupArray(cast(EventTime as UInt64)) as times,
        groupArray(URL) as urls,
        arrayFilter(time, url -> url = 'https://supermarket.ru/main', times, urls)[1] as step1_time,
        arrayFilter(time, url -> url = 'https://supermarket.ru/catalog' and step1_time != 0 and time >= step1_time, times, urls)[1] as step2_time,
        arrayFilter(time, url -> url = 'https://supermarket.ru/product' and step2_time != 0 and time >= step2_time, times, urls)[1] as step3_time,
        arrayFilter(time, url -> url = 'https://supermarket.ru/basket' and step3_time != 0 and time >= step3_time, times, urls)[1] as step4_time,
        arrayFilter(time, url -> url = 'https://supermarket.ru/finish_order' and step4_time != 0 and time >= step4_time, times, urls)[1] as step5_time
    FROM
        (SELECT * FROM metrica_data.hits ORDER BY ClientID, EventTime)
    GROUP BY Browser, ClientID
)
GROUP BY Browser
FORMAT TabSeparatedWithNames
'''

fdf = get_clickhouse_df(q)

In [157]:
fdf.head()

In [158]:
q = '''
create table metrica_data.funnels_by_bro Engine = Log as
SELECT
    Browser,
    countIf(step1_time != 0) as step1,
    countIf(step2_time != 0) as step2,
    countIf(step3_time != 0) as step3,
    countIf(step4_time != 0) as step4,
    countIf(step5_time != 0) as step5
FROM
(
    SELECT
        Browser,
        ClientID,
        groupArray(cast(EventTime as UInt64)) as times,
        groupArray(URL) as urls,
        arrayFilter(time, url -> url = 'https://supermarket.ru/main', times, urls)[1] as step1_time,
        arrayFilter(time, url -> url = 'https://supermarket.ru/catalog' and step1_time != 0 and time >= step1_time, times, urls)[1] as step2_time,
        arrayFilter(time, url -> url = 'https://supermarket.ru/product' and step2_time != 0 and time >= step2_time, times, urls)[1] as step3_time,
        arrayFilter(time, url -> url = 'https://supermarket.ru/basket' and step3_time != 0 and time >= step3_time, times, urls)[1] as step4_time,
        arrayFilter(time, url -> url = 'https://supermarket.ru/finish_order' and step4_time != 0 and time >= step4_time, times, urls)[1] as step5_time
    FROM
        (SELECT * FROM metrica_data.hits ORDER BY ClientID, EventTime)
    GROUP BY Browser, ClientID
)
GROUP BY Browser
'''

get_clickhouse_data(q)